# Interactive Dashboard with Ploty for Space X Launch Sites

In this part of our investigation, we generate an interactive dashboard. The app skeleton comprises the title, the dropdown menu where you will choose the launch site used by the Falcon 9 or select all to study the correlation between some variables, two graphs, and a range slider for the payload mass. The first graph will be a pie chart representing the success rate of each launch site. The second will be a scatter plot which will output the payload mass vs. the class (launch outcome). Each point on the scatter plot will mark the different booster versions used. 

This code has three components: the app layout and two call-back functions, each for the plots. 

Here are some of the findings: 
* KSC LC-39A is the launch site with the highest success rate of 77%
* Between 2k and 5.5k for the payload mass (kg) there seems to be the highest success rate
* The lowest success rate is above 5.5k for the payload mass (kg)
* The Falcon 9 booster version FT has the highest success rate

In [ ]:
# Uncomment if you dont have dash installed
# !pip3 install pandas dash

In [ ]:
import wget
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
# Download the data set
# !wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df.head()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
               
                # Add a dropdown list to enable Launch Site selection
                # The default select value is for ALL sites
                dcc.Dropdown(id='site-dropdown',
                            options=[
                                    {'label': 'All Sites', 'value': 'ALL'},
                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                    ],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                            ),

                html.Br(),

                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                html.Div(dcc.Graph(id='success-pie-chart')),
               
                html.Br(),

                html.P("Payload range (Kg):"),
                # Add a slider to select payload range
                dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=1000,
                                marks={0: '0', 100: '100'},
                                value=[min_payload, max_payload]),

                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                ])      


In [ ]:
# A callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Success Rate by Site')
        return fig
    else:
        data_df = filtered_df[filtered_df['Launch Site']==entered_site]
        perc = 1-data_df.loc[0,'class']
        data_df = data_df.append({'Launch Site': 'Failure', 'class': perc}, ignore_index=True)
        fig = px.pie(data_df, values='class', names='Launch Site', title='Success Rate for '+entered_site)
        return fig

# A callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_plot(entered_site, payload_mass):
    filtered_df = spacex_df[['Launch Site', 'Payload Mass (kg)', 'class', 'Booster Version Category']]
    filtered_df = filtered_df[filtered_df['Payload Mass (kg)']>payload_mass[0]]
    filtered_df = filtered_df[filtered_df['Payload Mass (kg)']<payload_mass[1]]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', symbol='Booster Version Category', title='Success Rate Percentage of All Sites')
        return fig
    else:
        data_df = filtered_df[filtered_df['Launch Site']==entered_site] 
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', symbol='Booster Version Category', title='Success Rate Percentage of '+str(entered_site))
        return fig
    
# Run the app
if __name__ == '__main__':
    app.run_server()